# Обработка данных для функциональности "Определение обсценной лексики"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Загрузка корпуса с обсценной лексикой

In [2]:
!wget https://raw.githubusercontent.com/odaykhovskaya/obscene_words_ru/master/obscene_corpus.txt

--2021-05-08 09:23:44--  https://raw.githubusercontent.com/odaykhovskaya/obscene_words_ru/master/obscene_corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152344 (149K) [text/plain]
Saving to: ‘obscene_corpus.txt’

obscene_corpus.txt  100%[===================>] 148.77K  --.-KB/s    in 0.03s   

2021-05-08 09:23:45 (5.41 MB/s) - ‘obscene_corpus.txt’ saved [152344/152344]



### Загрузка корпуса с 2ch

In [3]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/2ch_corpus.txt.zip

--2021-05-08 09:23:53--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/2ch_corpus.txt.zip
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/2ch_corpus.txt.zip [following]
--2021-05-08 09:23:54--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/2ch_corpus.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6255552 (6.0M) [application/zip]
Saving to: ‘2ch_corpus.txt.zip’

2ch_corpus.txt.zip  100%[===================>]   5.96M  24.4MB/s    in 0.2s    

2021-05-08 09:23:55 (24.4 MB/s) - ‘2ch_cor

In [4]:
!unzip 2ch_corpus.txt.zip

Archive:  2ch_corpus.txt.zip
  inflating: 2ch_corpus.txt          
  inflating: __MACOSX/._2ch_corpus.txt  


In [6]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 8.2MB 4.6MB/s 


### Импорты и пути

In [10]:
import sys
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from pymorphy2 import MorphAnalyzer
from tqdm import tqdm
from string import punctuation

tokenizer = nltk.tokenize.WordPunctTokenizer().tokenize
morph = MorphAnalyzer()

punct = punctuation+'«»—…“”*№–'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
obscene_data_path = "obscene_corpus.txt"
obscene_preprocessed_data_path = "/content/drive/MyDrive/thesis_datasets/obscene_preprocessed_corpus.txt"
not_obscene_data_path = "2ch_corpus.txt"
not_obscene_preprocessed_data_path = "/content/drive/MyDrive/thesis_datasets/2ch_corpus_preprocessed.txt"

## Функции для предобработки корпусов

Функция лемматизации с возможностью выбора токенайзера

In [9]:
def lemmatization_with_tokenizer(text: str, tokenizer) -> list:
    
    words = tokenizer(text)
    words = [morph.parse(word.lower().strip('\n').strip(punct))[0].normal_form for word in words]

    return words

Обработка корпуса обсценной лексики

In [11]:
def obscene_data_preprocess(data: list):
    
    data = [word.lower().strip('\n').strip(punct) for word in data]
    data = data
    return data

Обработка корпуса с 2ch

In [12]:
def not_obscene_data_lemmatization(data: list, tokenizer):
    
    """ tokenize and lemmatize texts """
    
    cleaned_data = []
    for text in tqdm(data):
        text = lemmatization_with_tokenizer(text, tokenizer)
        cleaned_data.append(text)
        
    return cleaned_data

In [13]:
def not_obscene_data_clean1(data: list, obscene_data: list): #all obscene words + additional words
    
    """ remove obscene words from texts """

    cleaned_data = []
    for text in tqdm(data):
        text = [word for word in text if word not in obscene_data]
        cleaned_data.append(text)
        
    return cleaned_data

In [14]:
def not_obscene_data_clean2(data: list, obscene_roots: list): #roots + parts - complicate cases
    
    """ remove words with obscene morphemes """
    all_words = []
    obscened_words = []
    for sentence in tqdm(data):
        for word in sentence:
            all_words.append(word)
            for obscene_root in obscene_roots:
                if obscene_root in word:
                    obscened_words.append(word)
    
    obscened_words = set(obscened_words)
    all_words = set(all_words)
    cleaned_data = all_words - obscened_words - {''}
    
    return list(cleaned_data), list(obscened_words)

## Предобработка корпуса обсценной лексики

In [15]:
with open(obscene_data_path, "r") as obscene_data_file:
    obscene_data = obscene_data_file.readlines()

In [16]:
obscene_data[:20]

['6ЛЯ\n',
 '6ЛЯД\n',
 'FUCK\n',
 'PIZD\n',
 'PIZDA\n',
 'PIZDAH\n',
 'PIZDAKH\n',
 'PIZDAM\n',
 'PIZDAMI\n',
 'PIZDAX\n',
 'PIZDE\n',
 'PIZDOI\n',
 'PIZDOJ\n',
 'PIZDOY\n',
 'PIZDU\n',
 'PIZDY\n',
 'SCHEISE\n',
 'SCHEISSE\n',
 'SHIT\n',
 'SUCK\n']

In [17]:
#obscene dataset = all obscene words + added words

additional_words = ['шлюх']

obscene_data = obscene_data + additional_words

Проверим, что в корпусе нет дубликатов

In [18]:
len(obscene_data)

7352

In [19]:
len(set(obscene_data))

7352

Подготовим отдельно список слов, которые могут быть обсценными корнями

In [20]:
#roots + parts - complicate cases

roots = [el for el in obscene_data if el.endswith("*\n")]
roots = obscene_data_preprocess(roots)
roots = set(roots)

parts = [el for el in obscene_data if len(el) < 5]
parts = obscene_data_preprocess(parts)
parts = set(parts)

complicate_cases = {'ёб',
                    'ёбс',
                    'ёбу',
                    'бл',
                    'еб',
                    'еба',
                    'еби',
                    'ебл',
                    'ебс',
                    'ебу',
                    'епт',
                    'еть',
                    'ипу',
                    'кал',
                    'куй',
                    'нах',
                    'пох',
                    'поц',
                    'фак',
                    'чмо'}

obscene_roots = sorted(list(roots | parts - complicate_cases))

In [21]:
len(obscene_roots)

49

In [22]:
obscene_data = obscene_data_preprocess(obscene_data)
obscene_data[:10]

['6ля',
 '6ляд',
 'fuck',
 'pizd',
 'pizda',
 'pizdah',
 'pizdakh',
 'pizdam',
 'pizdami',
 'pizdax']

In [44]:
len(obscene_data)

7352

In [23]:
with open(obscene_preprocessed_data_path, "w") as obscene_preprocessed_file:
    for word in obscene_data:
        obscene_preprocessed_file.write(word + "\n")

## Предобработка корпуса с 2ch

In [24]:
with open(not_obscene_data_path, "r") as not_obscene_data_file:
    not_obscene_data = not_obscene_data_file.readlines()

In [25]:
not_obscene_data[:10]

[" Анимублядский WebM-треддля приличных анимублядей и прочих аутистов. Безграмотное быдло с дубляжом, войсовером, порнографией и котиками, советы мерзких мокрописечников, вниманиебляди всех видов и прочее непотребство отправляется в порнотред <ссылка>.Для поиска сoуса видео сохраняем кадр (правый клик по видео) и ищем его на Для воспроизведения WebM с 10-битным цветом нужно установить плагин vlc ( ) и отключить встроенный в браузер плеер (media. webm. enabled=false в firefox).О кодировании WebMДоступные кодеки — VP8 и VP9 для видео, Vorbis и Opus для звука, максимальный размер файла — 10240КБ, всех файлов в посте — около 40МБ. Делать WebM можно научиться в вики треда: Там находится подробная информация о выборе и настройке кодеков на примерах использования консольных утилит ffmpeg, vpxenc и mkvmerge. Неочевидные моменты— libvorbis при указании битрейта (-b:a) работает в режиме CBR (постоянный битрейт), и это портит качество звука; для режима VBR вместо битрейта надо указывать качество 

In [26]:
len(not_obscene_data)

85903

In [27]:
%%time
not_obscene_data = not_obscene_data_lemmatization(not_obscene_data, tokenizer)

100%|██████████| 85903/85903 [08:46<00:00, 163.17it/s]

CPU times: user 8min 40s, sys: 3.39 s, total: 8min 43s
Wall time: 8min 46s


In [28]:
%%time
not_obscene_data = not_obscene_data_clean1(not_obscene_data, obscene_data)

100%|██████████| 85903/85903 [04:18<00:00, 332.33it/s]


CPU times: user 4min 15s, sys: 1.68 s, total: 4min 17s
Wall time: 4min 18s


In [29]:
%%time
not_obscene_data_cleaned, obscened_words = not_obscene_data_clean2(not_obscene_data, obscene_roots)

100%|██████████| 85903/85903 [00:05<00:00, 14417.32it/s]


CPU times: user 6.14 s, sys: 71.9 ms, total: 6.21 s
Wall time: 6.27 s


In [30]:
len(obscened_words)

1755

In [31]:
obscened_words = [word.strip(punct) for word in obscened_words]

In [32]:
obscened_words = sorted(list(set(obscened_words) - {'херсон', 'херсонский', 'хертстоун', 'батхерта', 'фехерваря', 'баттхертеть', 'целленхерер', 'хертс', 'парикмахерша', 'баттхерта', 'баттхерта?ть', 'баттхерта-тред', 'херсонщина', 'шухер', 'батхертнуть'}))

In [35]:
obscened_words[-10:]

['этотблядский',
 'эфироблядка',
 'ютубоблядь',
 'яблоблядок',
 'яблоблядь',
 'яебалпиздетьазазазатроллолололоть',
 'янихуянепонела',
 'япидорашкен',
 'ясенхуй',
 'ёбля']

In [45]:
len(obscened_words)

1742

In [36]:
with open(obscene_preprocessed_data_path, "a") as obscene_preprocessed_file:
    for word in obscened_words:
        obscene_preprocessed_file.write(word + "\n")

In [37]:
not_obscene_data_cleaned[:10]

['ударник',
 'шизаанальный',
 'гуляние',
 '153',
 'неопредленный',
 'преобладать',
 'теоретический',
 'знактого',
 'мысл',
 'хёвый']

In [38]:
len(not_obscene_data_cleaned)

95549

In [41]:
with open(not_obscene_preprocessed_data_path, "w") as not_obscene_preprocessed_file:
    for word in not_obscene_data_cleaned:
        not_obscene_preprocessed_file.write(word + "\n")

In [42]:
with open(not_obscene_preprocessed_data_path, "r") as not_obscene_data_file:
    not_obscene_data = not_obscene_data_file.readlines()

In [43]:
not_obscene_data[:20]

['ударник\n',
 'шизаанальный\n',
 'гуляние\n',
 '153\n',
 'неопредленный\n',
 'преобладать\n',
 'теоретический\n',
 'знактого\n',
 'мысл\n',
 'хёвый\n',
 'кантор\n',
 'срали\n',
 'декодер\n',
 'изолироваться\n',
 'العام\n',
 'insatiable\n',
 'недоразвитый\n',
 'приятель\n',
 'offensive\n',
 'отталкивающе\n']